In [27]:
import numpy as np
import matplotlib.pyplot as plt

In [28]:
def V_k(t):
    return 1 if np.cos(t * 150) > 0 else -1

In [29]:
# Define dimensions
n_x = 3
n_theta = 10

In [6]:
# Initialise terms
P = np.eye(n_x)
R = 0.1 * np.eye(n_x)
Q = 0.1 * np.eye(n_x)
Upsilon = np.zeros((n_x, n_theta))
S = np.eye(n_theta)

llama = 0.995
a = 0.999

In [8]:
# Initialise state and parameter
x_hat = np.zeros((n_x,1))
theta_hat = np.zeros((n_theta,1))
x_true = np.array([[1.0], [3.0]])  # Initial position and velocity

In [10]:
# Define priori
m = 23.8
X_u = -0.7225
X_uu = -1.3274
Y_v = -0.8612
Y_vv = -36.2823
Y_r = 0.1079
N_v = 0.1052
N_r = -0.5
N_rr = -1
X_ud = -2
Y_vd = -10
Y_rd = 0
N_vd = 0
N_rd = -1
I_z = 1.76
x_g = 0.046

In [13]:
m11 = m - X_ud
m22 = m - Y_vd
m23 = m * x_g - Y_rd
m32 = m * x_g - N_vd
m33 = I_z - N_rd
M = np.array([[m11, 0, 0], [0, m22, m23], [0, m32, m33 ]])

In [35]:
ts = 0.6
dt = 0.00001
N = int(ts/dt)
x_estimates = []
x_trues = []
theta_estimates = []
time = np.arange(N) * dt

In [37]:
B = np.array([[1, 0, 0],[0,0,1]])
mm = (m22 * m33) - (m23 * m32)

In [44]:
def C_k(v):
    C13 = -(m22*v[1])-((m23 + m32)/2)*v[2]
    C23 = m11 * v[0]
    C31 = (m22*v[1])+((m23 + m32)/2)*v[2]
    C32 = -m11 * v[0]
    return np.array([[0,0,C13], [0,0,C23], [C31, C32, 0]])

def B_tau_k(tau):
    btk = np.array([[tau[0], 0, tau[1]]])
    return btk.T

In [45]:
dt_int = -np.array([[dt/m11, 0,0,], [0, (dt*m33)/mm, -(dt*m23)/mm], [0,-(dt*m32)/mm, (dt*m22)/mm]])

In [47]:
def b_k(v, tau):
    return dt_int * (B_tau_k(tau) - (C_k(v)@v))

In [39]:
print(D_coeff)

[[-3.87596899e-07 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -2.99708719e-07  1.18884459e-07]
 [-0.00000000e+00  1.18884459e-07 -3.67034591e-06]]


In [22]:
for k in range(N):
    Fk = V_k(k * dt)
    Phi_k = np.array([
        [x_true[0, 0], x_true[1, 0], Fk, 0, 0, 0],
        [0, 0, 0, x_true[0, 0], x_true[1, 0], Fk]
    ])

    # Compute gains
    P = P + Q
    Sigma = P + R
    K = P @ np.linalg.inv(Sigma)
    P = (np.eye(n_x) - K) @ P


    Omega = Upsilon + Phi_k
    Upsilon = (np.eye(n_x) - K) @ Upsilon + (np.eye(n_x) - K) @ Phi_k
    Lambda = np.linalg.inv(llama * Sigma + Omega @ S @ Omega.T)
    Pi = S @ Omega.T @ Lambda
    S = (1 / llama) * S - (1/ llama) * S @ Omega.T @ Lambda @ Omega @ S

    # Estimate state and parameters
    y_k = x_true  # Assume perfect measurements (no noise)
    theta_previous = theta_hat.copy()
    innovation = y_k - (x_hat + Phi_k @ theta_previous)
    theta_hat = theta_previous + Pi @ innovation
    x_hat = x_hat + Phi_k @ theta_previous + K @ innovation + Upsilon @ (theta_hat - theta_previous)
    # theta_hat = theta_hat + theta_update
    
    # Update Covariance matrices
    Q = a * Q + (1 - a) * (K @ (innovation @ innovation.T) @ K.T)
    R = a * R + (1 - a) * (innovation @ innovation.T + P)

    # Store results
    x_estimates.append(x_hat.flatten())
    theta_estimates.append(theta_hat.flatten())

    # True system update
    x_true = x_true + np.array([
        [-r_true * dt/l_true, -k_true * dt/l_true],
        [k_true * dt/ j_true, -f_true * dt / j_true]
    ]) @ x_true + np.array([[dt / l_true], [0]]) * Fk

    x_trues.append(x_true.flatten())

ValueError: operands could not be broadcast together with shapes (2,10) (2,6) 

array([[25.8   ,  0.    ,  0.    ],
       [ 0.    , 33.8   ,  1.0948],
       [ 0.    ,  1.0948,  2.76  ]])